In [1]:
import os
directory = os.getcwd()  
directory = directory.split('src')
path = directory[0]
%cd $path
%pwd

c:\Users\JENMOSQU\Documents\PERSONAL\cod\Probabilistic-Electricity-Demand-


'c:\\Users\\JENMOSQU\\Documents\\PERSONAL\\cod\\Probabilistic-Electricity-Demand-'

In [2]:
# Libraries..

import numpy as np
import pandas as pd


# Plotly
import plotly.express as px
import plotly.graph_objects as go

# sklearn
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.neural_network import MLPRegressor
from sklearn.impute import KNNImputer

#skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster


# statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.formula.api as smf

# pmdarima
from pmdarima.arima import auto_arima

#scipy
from scipy.stats import norm

#Neuralprophet
# from neuralprophet import NeuralProphet

# Modules from repo
from src.Utilities import utilities as tool

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
# Configuración options.display
# ==============================================================================
pd.set_option('display.max_rows', 5000)
pd.options.display.max_rows = None
pd.options.display.max_columns = 5000
pd.options.display.max_colwidth = 5000
pd.options.display.max_dir_items = 5000
pd.options.display.width = 5000
pd.options.display.float_format = '{:.3f}'.format

c:\Users\JENMOSQU\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tensorflow WARNING From c:\Users\JENMOSQU\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



Using TensorFlow backend


## Skforecast

In [3]:
#Import Data..............

df=pd.read_parquet('df.parquet')
columnas_indeseadas =['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D'] #columna con correlación 0 con la demanda
columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(1, 29)]
# columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(7, 29, 7)]
print("columnas rezagadas que no son necesarias porque skforecast rezaga internamente",columnas_indeseadas)
df = df.loc[:, ~df.columns.isin(columnas_indeseadas)]
print(df.head(1))
print("--------------------------------------------------------------------------------------------------")
print(df.tail(1))

columnas rezagadas que no son necesarias porque skforecast rezaga internamente ['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D', 'Demanda_SIN_1', 'Demanda_SIN_2', 'Demanda_SIN_3', 'Demanda_SIN_4', 'Demanda_SIN_5', 'Demanda_SIN_6', 'Demanda_SIN_7', 'Demanda_SIN_8', 'Demanda_SIN_9', 'Demanda_SIN_10', 'Demanda_SIN_11', 'Demanda_SIN_12', 'Demanda_SIN_13', 'Demanda_SIN_14', 'Demanda_SIN_15', 'Demanda_SIN_16', 'Demanda_SIN_17', 'Demanda_SIN_18', 'Demanda_SIN_19', 'Demanda_SIN_20', 'Demanda_SIN_21', 'Demanda_SIN_22', 'Demanda_SIN_23', 'Demanda_SIN_24', 'Demanda_SIN_25', 'Demanda_SIN_26', 'Demanda_SIN_27', 'Demanda_SIN_28']
       FECHA TIPO_DIA  Demanda_SIN  Covid  Cambio_nivel  Tiempo  precipitacion  Temperatura  HRA2_MN_D  HRA2_MX_D  THSM_MEDIA_D  THSM_MX_D  VV_10_MEDIA_D  VV_10_VECT_MEDIA_D  THSM_MN_D  PT_2_TT_D  PT_2_MX_D  PT_10_MX_D  PT_10_TT_D  TSSM_MEDIA_D  TSSM_MN_D  TSSM_MX_D  PTPG_TT_D  BSHG_TT_D  TPR_CAL
0 2017-12-01  VIERNES   193865.750      0             0       1          9.589       22.203  

In [4]:
#Gráfica de Demanda SIN Nueva...................................

trace1 = go.Scatter(
    x=df['FECHA'],
    y=df['Demanda_SIN'],
    #mode='lines+markers',  # Esto agrega marcadores a la línea
    line=dict(shape='linear', color='#276981'), 
    name='Demanda_SIN' 
)
fig = go.Figure(data=[trace1])
fig.update_layout(
    legend=dict(
        title='Series',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)
fig.update_xaxes(title='Date-daily')
fig.update_yaxes(title='Electricity Demand SIN kWh')
fig.show()

In [5]:
#Correlación variables y filtrado........
#['THSM_MEDIA_D','PTPG_TT_D']
correlacion_con_demanda = df.corr()['Demanda_SIN']
nombres_variables = correlacion_con_demanda[(correlacion_con_demanda >= 0.2) | (correlacion_con_demanda <= -0.2)]
print(nombres_variables)
nombres_variables=nombres_variables.index.tolist()
nombres_variables +=['THSM_MEDIA_D'] #'PTPG_TT_D' #'VVAG_MEDIA_D'
nombres_variables.remove('Demanda_SIN')
df=df[['FECHA', 'TIPO_DIA', 'Demanda_SIN'] + nombres_variables]
print("columnas finales", df.columns)
df.head(2)

Demanda_SIN           1.000
Covid                -0.246
Cambio_nivel          0.352
Tiempo                0.595
VV_10_MEDIA_D         0.213
VV_10_VECT_MEDIA_D    0.210
PT_10_TT_D           -0.291
Name: Demanda_SIN, dtype: float64
columnas finales Index(['FECHA', 'TIPO_DIA', 'Demanda_SIN', 'Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D'], dtype='object')


,FECHA,TIPO_DIA,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D
0,2017-12-01,VIERNES,193865.750,0,0,1,1.703,0.935,6.232,19.113
1,2017-12-02,SABADO,184393.750,0,0,2,1.757,0.930,7.888,19.166


In [6]:
#Imputation: Rolling mean........................

columns_rolling = ['VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D','PT_10_TT_D' ,'THSM_MEDIA_D']
window_size=60
df = tool.exploratory_tools.fill_null_values_with_rolling_mean(df, columns_rolling, window_size)

In [7]:
# Transformer one hot encoding and formatting-.......

print("--------------------------------------------------------------------------------------------------------------------------------------")
# ==============================================================================
one_hot_encoder = make_column_transformer(
                      #(StandardScaler(), ['Demanda_SIN']),
                      (   
                          OneHotEncoder(sparse_output=False, drop='if_binary'),
                          ['TIPO_DIA'],
                      ),
                      remainder="passthrough",
                      verbose_feature_names_out=False,
                  ).set_output(transform="pandas")

df = one_hot_encoder.fit_transform(df)

#-------------------------------------------------------------------------------------------------------------------------------------------
#Variable TIPO_DIA
#-------------------------------------------------------------------------------------------------------------------------------------------
columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = ['FECHA'] + nombres_variables + columnas_tipo_dia
variable_dependiente = ['Demanda_SIN']
df = df[variable_dependiente + variables_exogenas] 

#-------------------------------------------------------------------------------------------------------------------------------------------
# Codificación de Fecha...
#-------------------------------------------------------------------------------------------------------------------------------------------
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d')
df = df.set_index('FECHA')
df = df.asfreq('D')


df.head(2)

--------------------------------------------------------------------------------------------------------------------------------------


,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D,TIPO_DIA_1--ENE,TIPO_DIA_1--MAY,TIPO_DIA_2--ENE,TIPO_DIA_20--JUL,TIPO_DIA_24--DIC,TIPO_DIA_25--DIC,TIPO_DIA_31--DIC,TIPO_DIA_7--AGO,TIPO_DIA_8--DIC,TIPO_DIA_DOALF,TIPO_DIA_DOALFENE,TIPO_DIA_DOMINGO,TIPO_DIA_DOVDIC,TIPO_DIA_DOVENE,TIPO_DIA_DSS,TIPO_DIA_JSS,TIPO_DIA_JUEVES,TIPO_DIA_JUVDIC,TIPO_DIA_JUVENE,TIPO_DIA_LF,TIPO_DIA_LFENE,TIPO_DIA_LUNES,TIPO_DIA_LUVDIC,TIPO_DIA_LUVENE,TIPO_DIA_MADLF,TIPO_DIA_MARTES,TIPO_DIA_MAVDIC,TIPO_DIA_MAVENE,TIPO_DIA_MIERCOLES,TIPO_DIA_MIVDIC,TIPO_DIA_MIVENE,TIPO_DIA_MSS,TIPO_DIA_SAALF,TIPO_DIA_SAALFENE,TIPO_DIA_SABADO,TIPO_DIA_SAVDIC,TIPO_DIA_SAVENE,TIPO_DIA_SSS,TIPO_DIA_VIERNES,TIPO_DIA_VIVDIC,TIPO_DIA_VIVENE,TIPO_DIA_VSS
FECHA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-12-01,193865.750,0,0,1,1.703,0.935,6.232,19.113,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000
2017-12-02,184393.750,0,0,2,1.757,0.930,7.888,19.166,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [8]:
# Lista de variables a graficar Precipitación y temperatura..............
variables = ['PT_10_TT_D', 'THSM_MEDIA_D']

# Crear un gráfico de líneas interactivas
fig = px.line(df, x=df.index, y=variables,
              title='Daily Meteorological Variables',
              labels={'value': 'Value', 'index': 'Date'},
              color_discrete_map={'PT_10_TT_D': '#276981', 'THSM_MEDIA_D': '#96b1be'})

# Personalizar el diseño del gráfico
fig.update_layout(
    legend=dict(
        title='Variables',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    xaxis_title='Date-daily',
    yaxis_title='Value',
)

# Mostrar el gráfico interactivo
fig.show()


In [8]:
#Velocidad media y Persistencia diaria

# Lista de variables a graficar
variables = ['VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D']

# Crear un gráfico de líneas interactivas
fig = px.line(df, x=df.index, y=variables,
              title='Daily Meteorological Variables',
              labels={'value': 'Value', 'index': 'Date'},
              color_discrete_map={'VV_10_MEDIA_D': '#276981', 'VV_10_VECT_MEDIA_D': '#96b1be'})

# Personalizar el diseño del gráfico
fig.update_layout(
    legend=dict(
        title='Variables',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    xaxis_title='Date-daily',
    yaxis_title='Value',
)

# Mostrar el gráfico interactivo
fig.show()

In [9]:
# Separación datos train-val-test................
#-------------------------------------------------------------------------------------------------------------------------------------------

datos = df[df.index >= '2017-12-01']
fin_train = '2023-05-31'
fin_validacion = '2023-09-23'

datos_train = datos[datos.index <= fin_train].copy()
datos_val = datos[(datos.index > fin_train) & (datos.index <= fin_validacion)].copy()
datos_test = datos[datos.index > fin_validacion].copy()

print(f"Fechas train      : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
print(f"Fechas validacion : {datos_val.index.min()} --- {datos_val.index.max()}  (n={len(datos_val)})")
print(f"Fechas test       : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")
print("-------------------------------------------------------------------------------------------------------------------------------------")

datos_test.head(2)


Fechas train      : 2017-12-01 00:00:00 --- 2023-05-31 00:00:00  (n=2008)
Fechas validacion : 2023-06-01 00:00:00 --- 2023-09-23 00:00:00  (n=115)
Fechas test       : 2023-09-24 00:00:00 --- 2023-09-30 00:00:00  (n=7)
-------------------------------------------------------------------------------------------------------------------------------------


,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D,TIPO_DIA_1--ENE,TIPO_DIA_1--MAY,TIPO_DIA_2--ENE,TIPO_DIA_20--JUL,TIPO_DIA_24--DIC,TIPO_DIA_25--DIC,TIPO_DIA_31--DIC,TIPO_DIA_7--AGO,TIPO_DIA_8--DIC,TIPO_DIA_DOALF,TIPO_DIA_DOALFENE,TIPO_DIA_DOMINGO,TIPO_DIA_DOVDIC,TIPO_DIA_DOVENE,TIPO_DIA_DSS,TIPO_DIA_JSS,TIPO_DIA_JUEVES,TIPO_DIA_JUVDIC,TIPO_DIA_JUVENE,TIPO_DIA_LF,TIPO_DIA_LFENE,TIPO_DIA_LUNES,TIPO_DIA_LUVDIC,TIPO_DIA_LUVENE,TIPO_DIA_MADLF,TIPO_DIA_MARTES,TIPO_DIA_MAVDIC,TIPO_DIA_MAVENE,TIPO_DIA_MIERCOLES,TIPO_DIA_MIVDIC,TIPO_DIA_MIVENE,TIPO_DIA_MSS,TIPO_DIA_SAALF,TIPO_DIA_SAALFENE,TIPO_DIA_SABADO,TIPO_DIA_SAVDIC,TIPO_DIA_SAVENE,TIPO_DIA_SSS,TIPO_DIA_VIERNES,TIPO_DIA_VIVDIC,TIPO_DIA_VIVENE,TIPO_DIA_VSS
FECHA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-09-24,203949.278,0,1,2124,49.364,49.926,5.819,20.972,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2023-09-25,233453.085,0,1,2125,49.389,49.956,5.880,20.969,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [61]:
# # Gráfico de demanda..

# # Crear un gráfico de serie temporal con Plotly
# fig = px.line(datos, x=datos.index, y='Demanda_SIN', color=datos.index.year,
#               title='Demanda eléctrica',
#               labels={'Demanda_SIN': 'Demanda'})

# # Añadir etiquetas de las particiones de datos (entrenamiento, validación, test)
# fig.add_shape(type="rect", x0=datos_train.index[0], x1=datos_train.index[-1],
#               y0=0, y1=1, line=dict(color="blue", width=2),
#               xref='x', yref='paper', fillcolor="blue", opacity=0.3,
#               name='entrenamiento')

# fig.add_shape(type="rect", x0=datos_val.index[0], x1=datos_val.index[-1],
#               y0=0, y1=1, line=dict(color="orange", width=2),
#               xref='x', yref='paper', fillcolor="orange", opacity=0.3,
#               name='validación')

# fig.add_shape(type="rect", x0=datos_test.index[0], x1=datos_test.index[-1],
#               y0=0, y1=1, line=dict(color="green", width=2),
#               xref='x', yref='paper', fillcolor="green", opacity=0.3,
#               name='test')

# # Mostrar el gráfico
# fig.show()


# # # Gráfico serie temporal
# # # ==============================================================================
# # fig, ax = plt.subplots(figsize=(8, 5))
# # datos_train.Demanda_SIN.plot(ax=ax, label='entrenamiento', linewidth=1)
# # datos_val.Demanda_SIN.plot(ax=ax, label='validación', linewidth=1)
# # datos_test.Demanda_SIN.plot(ax=ax, label='test', linewidth=1)
# # ax.set_title('Demanda eléctrica')
# # ax.legend();

In [ ]:
# #Grilla de valores.........
# gridz = 10 ** np.linspace(-2, 10, num=100)
# print("valor mínimo",gridz.min())
# print("valor máximo",gridz.max())
# # Crear un histograma con Plotly
# fig = px.histogram(x=gridz, nbins=20, log_y=True,
#                    labels={'x': 'Valor', 'y': 'Frecuencia'},
#                    title='Histograma de 10 ** np.linspace(-2, 10, num=100)')
# fig.show()



# import random
# random.seed(123)
# #random.sample(range(100), 10)
# #random.sample(range(20), 5)
# np.logspace(-10, 3, 100).max()
# np.linspace(0.001, 1.0, num=300)
# np.linspace(0.0001, 1.0, num=10)


# data = np.logspace(-5, 2, 200)
# fig = px.histogram(x=data, nbins=20, log_y=True,
#                    labels={'x': 'Valor', 'y': 'Frecuencia'},
#                    title='Histograma de np.logspace(-10, 3, 200)')
# fig.show()

# print("Minímo: 0.0000000001:",np.logspace(-10, 3, 200).min() )
# #min: 0.0000000001
# print("Máximo",np.logspace(-10, 3, 200).max())

In [10]:
# Sin búsqueda de parámetros.........

# ForecasterAutoreg: Ridge
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor     = Ridge(alpha=0.3, random_state=123),
                 lags          = [1, 2, 3, 4, 5, 6, 7, 14,21, 28],
                 transformer_y = StandardScaler()
             )
# MAPE: 1.6111418829954176
#Error de backtest: 3607.9930242858914

# ForecasterAutoreg: Lasso
# ==============================================================================
# forecaster = ForecasterAutoreg(
#     regressor=Lasso(alpha=0.02, random_state=123), 
#     lags=[1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
#     transformer_y=StandardScaler()
# )
# Error de backtest: 5148.39846044951
# MAPE: 2.299006770286391

# # ForecasterAutoreg: XGBRegressor
# # ==============================================================================
# forecaster = ForecasterAutoreg(
#                 regressor = XGBRegressor(random_state=123),
#                 lags=[1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
#                 transformer_y = StandardScaler()
#              )
# Error de backtest: 4273.889835360962
# MAPE: 1.908496737856446

# ForecasterAutoreg: LGBMRegressor
# ==============================================================================
# forecaster = ForecasterAutoreg(
#                 regressor = LGBMRegressor(random_state=123),
#                 lags=[1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
#                 transformer_y = StandardScaler()
#              )
#MAPE: 2.0710136700119364

# ForecasterAutoreg: SVR
# ==============================================================================
# forecaster = ForecasterAutoreg(
#                 regressor = SVR(),
#                 lags=[1, 2, 3, 4, 5, 6, 7, 14,21, 28],
#                 transformer_y = StandardScaler()
#              )
# Error de backtest: 11317.396979678644
# MAPE: 5.053760403002729


# ForecasterAutoreg: GradientBoostingRegressor
# ==============================================================================
# forecaster = ForecasterAutoreg(
#                 regressor = GradientBoostingRegressor(random_state=123),
#                 lags=[1, 2, 3, 4, 5, 6, 7, 14,21, 28],
#                 transformer_y = StandardScaler()
#              )
# Error de backtest: 5033.759316053118
# MAPE: 2.247814895544796

columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = nombres_variables + columnas_tipo_dia

forecaster.fit(y=df.loc[:fin_validacion, 'Demanda_SIN'], exog=df.loc[:fin_validacion, variables_exogenas])

#Sin Búsqueda de hiperparametros
# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False, #Incremental
                            allow_incomplete_fold =True,
                            steps              = 7, #len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )
print(metric)
print(f"Error de backtest_mean_squared_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_absolute_percentage_error: {metric[2]:.2f}")
print(f"Error de backtest_mean_squared_log_error: {metric[3]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())


#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()


Information of backtesting process
----------------------------------
Number of observations used for initial training: 2008
Number of observations used for backtesting: 115
    Number of folds: 17
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-05-31 00:00:00  (n=2008)
    Validation: 2023-06-01 00:00:00 -- 2023-06-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-01 00:00:00 -- 2023-06-07 00:00:00  (n=2015)
    Validation: 2023-06-08 00:00:00 -- 2023-06-14 00:00:00  (n=7)
Fold: 2
    Training:   2017-12-01 00:00:00 -- 2023-06-14 00:00:00  (n=2022)
    Validation: 2023-06-15 00:00:00 -- 2023-06-21 00:00:00  (n=7)
Fold: 3
    Training:   2017-12-01 00:00:00 -- 2023-06-21 00:00:00  (n=2029)
    Validation: 2023-06-22 00:00:00 -- 2023-06-28 00:00:00  (n=7)
Fold: 4
    Training:   2017-12-01 00:00:00 -- 2023-06-28 00:00:00  (n

100%|██████████| 17/17 [00:33<00:00,  1.96s/it]

[22256183.43685964, 3583.272773009894, 0.01613807379966264, 0.0004554713305679164]
Error de backtest_mean_squared_error: 22256183.44
Error de backtest_mean_absolute_error: 3583.27
Error de backtest_mean_absolute_percentage_error: 0.02
Error de backtest_mean_squared_log_error: 0.00
MAPE: 1.6130732915609836
               feature  importance
0                lag_1       0.575
1                lag_2      -0.021
2                lag_3       0.039
3                lag_4       0.037
4                lag_5       0.047
5                lag_6       0.032
6                lag_7       0.063
7               lag_14       0.012
8               lag_21       0.023
9               lag_28       0.008
10               Covid      -0.052
11        Cambio_nivel      -0.118
12              Tiempo       0.000
13       VV_10_MEDIA_D       0.002
14  VV_10_VECT_MEDIA_D      -0.002
15          PT_10_TT_D      -0.010
16        THSM_MEDIA_D       0.075
17     TIPO_DIA_1--ENE      -1.354
18     TIPO_DIA_1--MAY      

## Ridge, Lasso,  ElasticNet

In [13]:
# Models to compare, Ridge,  ElasticNet.....
import random
random.seed(123)

models = [
          Ridge(random_state=123),
          ElasticNet(random_state=123)
          ]

# Hyperparameter to search for each model
param_grids = {
               'Ridge': {'alpha': np.logspace(-10, 5, 300)},
               'ElasticNet':{'alpha': np.linspace(0.001, 1.0, num=50), 'l1_ratio':np.linspace(0, 1, num=50)}
               }

                        
# Lags used as predictors
lags_grid = [6, 7 ,14, 21, 28]

columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = nombres_variables + columnas_tipo_dia

df_results = pd.DataFrame()
for i, model in enumerate(models):
    print(f"-------------------------")
    print(f"Grid search for regressor: {model}")
    print(f"-------------------------")

    forecaster = ForecasterAutoreg(
                     regressor = model,
                     lags      = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
                     transformer_y = StandardScaler()
                 )

    # Regressor hyperparameters
    param_grid = param_grids[list(param_grids)[i]]

    results_grid = grid_search_forecaster(
                       forecaster         = forecaster,
                       y                  = datos.loc[:fin_validacion, 'Demanda_SIN'],
                       exog               = datos.loc[:fin_validacion, variables_exogenas],
                       param_grid         = param_grid,
                       lags_grid          = lags_grid,
                       steps              = len(datos.loc[fin_train:fin_validacion]),
                       refit              = True, #True
                       metric             = ['mean_absolute_percentage_error','mean_squared_error', 'mean_absolute_error', 'mean_squared_log_error'],
                       initial_train_size = len(datos.loc[:fin_train]),
                       fixed_train_size   = True, # False
                       allow_incomplete_fold =True,
                       return_best        = True, #True
                       n_jobs             = 'auto',
                       verbose            = False, #True
                       show_progress      = True
                   )
    
    # Create a column with model name
    results_grid['model'] = list(param_grids)[i]
    
    df_results = pd.concat([df_results, results_grid])

df_results = df_results.sort_values(by='mean_absolute_percentage_error')



# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False, #Incremental
                            allow_incomplete_fold =True,
                            steps              = 7, #len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_absolute_percentage_error', 'mean_squared_error', 'mean_absolute_error', 'mean_squared_log_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )
print(metric)
print(f"Error de backtest_mean_absolute_percentage_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_squared_error: {metric[2]:.2f}")
print(f"Error de backtest_mean_squared_log_error: {metric[3]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
mape_m=mean_absolute_percentage_error(y_true, predicciones['pred'])
print("MAPE_METRICA",mape_m)
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())


#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()

-------------------------
Grid search for regressor: Ridge(random_state=123)
-------------------------
Number of models compared: 2500.


lags grid:   0%|          | 0/5 [00:00<?, ?it/s]

lags grid: 100%|██████████| 5/5 [01:15<00:00, 15.16s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
  Parameters: {'alpha': 1e-10}
  Backtesting metric: 0.019593624186446883

-------------------------
Grid search for regressor: ElasticNet(random_state=123)
-------------------------
Number of models compared: 32000.


lags grid: 100%|██████████| 5/5 [22:59<00:00, 275.82s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
  Parameters: {'alpha': 0.001, 'l1_ratio': 0.0}
  Backtesting metric: 0.019740119753642028

Information of backtesting process
----------------------------------
Number of observations used for initial training: 2008
Number of observations used for backtesting: 115
    Number of folds: 17
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-05-31 00:00:00  (n=2008)
    Validation: 2023-06-01 00:00:00 -- 2023-06-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-01 00:00:00 -- 2023-06-07 00:00:00  (n=2015)
    Validation: 2023-06-08 00:00:00 -- 2023-06-14 00:00:00  (n=7)
Fold: 2
    Training:   2017-12-01 00:00:00 -- 2023-06-14 00:00:00  (n=2022)
    Validation: 2023-06-15 00:00:00 -- 2023-0

100%|██████████| 17/17 [00:29<00:00,  1.71s/it]

[0.015929427330616457, 20740280.675700564, 3546.991986373098, 0.00042119929543441606]
Error de backtest_mean_squared_error: 0.02
Error de backtest_mean_absolute_error: 20740280.68
Error de backtest_mean_absolute_percentage_error: 3546.99
Error de backtest_mean_squared_log_error: 0.00
MAPE_METRICA 0.015929427330616457
MAPE: 1.5967408570442894
               feature  importance
0                lag_1       0.596
1                lag_2      -0.049
2                lag_3       0.030
3                lag_4       0.035
4                lag_5       0.040
5                lag_6       0.028
6                lag_7       0.136
7                lag_8      -0.061
8                lag_9      -0.011
9               lag_10       0.005
10              lag_11       0.009
11              lag_12      -0.016
12              lag_13       0.007
13              lag_14       0.037
14               Covid      -0.061
15        Cambio_nivel      -0.137
16              Tiempo       0.000
17       VV_10_MEDIA_D    

In [14]:
#MAPE on data_test.........
mape_data_test_Ridge=mean_absolute_percentage_error(datos_test['Demanda_SIN'].values, forecaster.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values)
print("MAPE..........................................................",mape_data_test_Ridge*100)

#save_forecaster(forecaster, file_name='Ml_models/forecaster_Ridge.py', verbose=False)

MAPE.......................................................... 1.0866156119298709


In [15]:
#MAPE on data_test.........
mape_data_test_elasticNet=mean_absolute_percentage_error(datos_test['Demanda_SIN'].values, forecaster.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values)
print("MAPE..........................................................",mape_data_test_elasticNet*100)

#save_forecaster(forecaster, file_name='Ml_models/forecaster_ElasticNet.py', verbose=False)

MAPE.......................................................... 1.0866156119298709


In [29]:
# Models to compare Lasso.....
import random
random.seed(123)

models = [
          Lasso(random_state=123)
          ]

# Hyperparameter to search for each model
param_grids = {
               'Lasso': {'alpha': np.logspace(-10, 5, 50)}
               }

                        
# Lags used as predictors
lags_grid = [6, 7 ,14, 21, 28]

columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = nombres_variables + columnas_tipo_dia

df_results = pd.DataFrame()
for i, model in enumerate(models):
    print(f"-------------------------")
    print(f"Grid search for regressor: {model}")
    print(f"-------------------------")

    forecaster = ForecasterAutoreg(
                     regressor = model,
                     lags      = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
                     transformer_y = StandardScaler()
                 )

    # Regressor hyperparameters
    param_grid = param_grids[list(param_grids)[i]]

    results_grid = grid_search_forecaster(
                       forecaster         = forecaster,
                       y                  = datos.loc[:fin_validacion, 'Demanda_SIN'],
                       exog               = datos.loc[:fin_validacion, variables_exogenas],
                       param_grid         = param_grid,
                       lags_grid          = lags_grid,
                       steps              = len(datos.loc[fin_train:fin_validacion]),
                       refit              = True, #True
                       metric             = ['mean_absolute_percentage_error','mean_squared_error', 'mean_absolute_error', 'mean_squared_log_error'],
                       initial_train_size = len(datos.loc[:fin_train]),
                       fixed_train_size   = True, # False
                       allow_incomplete_fold =True,
                       return_best        = True, #True
                       n_jobs             = 'auto',
                       verbose            = False, #True
                       show_progress      = True
                   )
    
    # Create a column with model name
    results_grid['model'] = list(param_grids)[i]
    
    df_results = pd.concat([df_results, results_grid])

df_results = df_results.sort_values(by='mean_absolute_percentage_error')



# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False, #Incremental
                            allow_incomplete_fold =True,
                            steps              = 7, #len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_absolute_percentage_error', 'mean_squared_error', 'mean_absolute_error', 'mean_squared_log_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )
print(metric)
print(f"Error de backtest_mean_absolute_percentage_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_squared_error: {metric[2]:.2f}")
print(f"Error de backtest_mean_squared_log_error: {metric[3]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
mape_m=mean_absolute_percentage_error(y_true, predicciones['pred'])
print("MAPE_METRICA",mape_m)
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())


#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()


-------------------------
Grid search for regressor: Lasso(random_state=123)
-------------------------
Number of models compared: 250.


lags grid:   0%|          | 0/5 [00:00<?, ?it/s]

lags grid: 100%|██████████| 5/5 [00:33<00:00,  6.75s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
  Parameters: {'alpha': 1.5998587196060572e-05}
  Backtesting metric: 0.019562778471020465

Information of backtesting process
----------------------------------
Number of observations used for initial training: 2008
Number of observations used for backtesting: 115
    Number of folds: 17
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-05-31 00:00:00  (n=2008)
    Validation: 2023-06-01 00:00:00 -- 2023-06-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-01 00:00:00 -- 2023-06-07 00:00:00  (n=2015)
    Validation: 2023-06-08 00:00:00 -- 2023-06-14 00:00:00  (n=7)
Fold: 2
    Training:   2017-12-01 00:00:00 -- 2023-06-14 00:00:00  (n=2022)
    Validation: 2023-06-15 00:00:00 -- 2023-0

100%|██████████| 17/17 [00:15<00:00,  1.07it/s]

[0.01585957732430891, 21372590.067813534, 3526.4261828620565, 0.00043560392494794187]
Error de backtest_mean_absolute_percentage_error: 0.02
Error de backtest_mean_absolute_error: 21372590.07
Error de backtest_mean_squared_error: 3526.43
Error de backtest_mean_squared_log_error: 0.00
MAPE_METRICA 0.01585957732430891
MAPE: 1.5874827987091749
               feature  importance
0                lag_1       0.569
1                lag_2      -0.013
2                lag_3       0.041
3                lag_4       0.038
4                lag_5       0.049
5                lag_6       0.029
6                lag_7       0.073
7                lag_8      -0.029
8                lag_9       0.011
9               lag_10       0.010
10              lag_11       0.009
11              lag_12      -0.015
12              lag_13       0.015
13              lag_14       0.017
14               Covid      -0.056
15        Cambio_nivel      -0.125
16              Tiempo       0.000
17       VV_10_MEDIA_D     

In [8]:
forecaster  = load_forecaster("Ml_models/forecaster_lasso.py")

ForecasterAutoreg 
Regressor: Lasso(alpha=1.821885343465165e-05, random_state=123) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] 
Transformer for y: StandardScaler() 
Transformer for exog: None 
Window size: 21 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D', 'TIPO_DIA_1--ENE', 'TIPO_DIA_1--MAY', 'TIPO_DIA_2--ENE', 'TIPO_DIA_20--JUL', 'TIPO_DIA_24--DIC', 'TIPO_DIA_25--DIC', 'TIPO_DIA_31--DIC', 'TIPO_DIA_7--AGO', 'TIPO_DIA_8--DIC', 'TIPO_DIA_DOALF', 'TIPO_DIA_DOALFENE', 'TIPO_DIA_DOMINGO', 'TIPO_DIA_DOVDIC', 'TIPO_DIA_DOVENE', 'TIPO_DIA_DSS', 'TIPO_DIA_JSS', 'TIPO_DIA_JUEVES', 'TIPO_DIA_JUVDIC', 'TIPO_DIA_JUVENE', 'TIPO_DIA_LF', 'TIPO_DIA_LFENE', 'TIPO_DIA_LUNES', 'TIPO_DIA_LUVDIC', 'TIPO_DIA_LUVENE', 'TIPO_DIA_MADLF', 'TIPO_D

In [30]:
#MAPE on data_test.........
mape_data_test_lasso=mean_absolute_percentage_error(datos_test['Demanda_SIN'].values, forecaster.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values)
print("MAPE..........................................................",mape_data_test_lasso*100)

#save_forecaster(forecaster, file_name='Ml_models/forecaster_lasso.py', verbose=False)

MAPE.......................................................... 1.0275777560505976


In [44]:
# Gráfico Predicciones del Intervals Forecaster with Boostrap...
# ==============================================================================
df = datos.loc[fin_train:fin_validacion, ['Demanda_SIN']].copy()
df['predictions'] = predicciones['pred']
df['lower_bound'] = predicciones['lower_bound']
df['upper_bound'] = predicciones['upper_bound']

fig = go.Figure()

# Agrega las series de datos con colores específicos

fig.add_trace(go.Scatter(x=df.index, y=df['Demanda_SIN'], mode='lines', name='Real Data', line=dict(color='#bc8f8f')))
fig.add_trace(go.Scatter(x=df.index, y=df['predictions'], mode='lines', name='Predictions', line=dict(color='#276981')))
fig.add_trace(go.Scatter(x=df.index, y=df['lower_bound'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', showlegend=False))
fig.add_trace(go.Scatter(x=df.index, y=df['upper_bound'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', name='Prediction Interval'))

# Estilo y diseño del gráfico
fig.update_layout(
    #title='Energy Demand Forecast with Prediction Interval',
    yaxis_title='Demand Forecast with Prediction Interval',
    xaxis_title='Date-daily',
    showlegend=True
)
# Mostrar el gráfico
fig.show()

In [52]:
# Interval coverage on test data
# ==============================================================================
inside_interval = np.where(
                      (datos.loc[predicciones.index, 'Demanda_SIN'] >= predicciones['lower_bound']) & \
                      (datos.loc[predicciones.index, 'Demanda_SIN'] <= predicciones['upper_bound']),
                      True,
                      False
                  )

coverage = inside_interval.mean()
print(f"Coverage of the predicted interval on test data: {100 * coverage}")

Coverage of the predicted interval on test data: 84.34782608695653


In [51]:
from sklearn.metrics import mean_pinball_loss
df.dropna(inplace=True)
mean_pinball_loss(df['Demanda_SIN'], df['predictions'])

1774.517930521855

In [59]:
# Prediction Intervals Test..............
# Predict intervals for next 7 steps, quantiles 10th and 90th
# ==============================================================================
predictions = forecaster.predict_interval(steps=len(datos_test), interval=[10, 90], n_boot=1000, exog=datos_test[variables_exogenas])
predictions

,pred,lower_bound,upper_bound
2023-09-24,205227.933,201525.328,208032.358
2023-09-25,230509.498,226146.446,233724.306
2023-09-26,233624.454,229528.894,236744.624
2023-09-27,234261.320,230085.730,237464.804
2023-09-28,235379.811,230701.488,238876.106
2023-09-29,235465.331,230874.255,239095.187
2023-09-30,225154.200,220185.397,228462.696


In [54]:
# Gráfico Predicciones del Intervals Forecaster with Boostrap...
# ==============================================================================
df = datos.loc[fin_validacion:, ['Demanda_SIN']].copy()
df = df.iloc[1:]
df['predictions'] = predictions['pred']
df['lower_bound'] = predictions['lower_bound']
df['upper_bound'] = predictions['upper_bound']

fig = go.Figure()

# Agrega las series de datos con colores específicos

fig.add_trace(go.Scatter(x=df.index, y=df['Demanda_SIN'], mode='lines', name='Real Data', line=dict(color='#bc8f8f')))
fig.add_trace(go.Scatter(x=df.index, y=df['predictions'], mode='lines', name='Predictions', line=dict(color='#276981')))
fig.add_trace(go.Scatter(x=df.index, y=df['lower_bound'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', showlegend=False))
fig.add_trace(go.Scatter(x=df.index, y=df['upper_bound'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', name='Prediction Interval'))

# Estilo y diseño del gráfico
fig.update_layout(
    #title='Energy Demand Forecast with Prediction Interval',
    yaxis_title='Demand Forecast with Prediction Interval',
    xaxis_title='Date-daily',
    showlegend=True
)
# Mostrar el gráfico
fig.show()

In [55]:
#datos_test
#...............................................................................
# Predict 10 different forecasting sequences of len(datos_test) steps each using bootstrapping
# ==============================================================================
boot_predictions = forecaster.predict_bootstrapping(steps=len(datos_test), n_boot=10, exog=datos_test[variables_exogenas])
boot_predictions


# from skforecast.plot import plot_prediction_distribution
# # Ridge plot of bootstrapping predictions
# # ==============================================================================
# _ = plot_prediction_distribution(boot_predictions, figsize=(7, 4))

,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9
2023-09-24,203939.409,206439.048,208039.799,205421.602,203581.915,195425.231,205140.720,202457.196,207337.673,206454.510
2023-09-25,236498.954,229959.932,219917.713,227371.991,233194.065,222675.835,233320.319,231025.410,232420.063,232838.831
2023-09-26,238242.462,228154.942,227504.061,230645.237,238447.009,225642.703,237052.674,236620.966,230015.754,236307.387
2023-09-27,236946.730,227211.315,232893.075,232980.621,236190.440,230929.657,231144.770,246817.668,238984.452,230437.465
2023-09-28,238944.208,235558.347,234978.338,234666.800,242183.279,228811.185,233993.170,239865.073,241066.456,231813.329
2023-09-29,235606.205,229774.998,234733.948,229814.924,240430.399,230860.122,237523.959,238424.455,238107.054,233844.045
2023-09-30,226652.828,220010.920,223100.015,223432.613,225558.622,221191.676,227526.251,224962.125,226475.572,227175.250


In [60]:
# Predict the parameters of a normal distribution for the next 120 steps
# ==============================================================================
predictions_dist = forecaster.predict_dist(steps=len(datos_test), distribution=norm, n_boot=1000, exog=datos_test[variables_exogenas])
predictions_dist

,loc,scale
2023-09-24,204922.827,2879.719
2023-09-25,230070.223,3153.492
2023-09-26,233185.107,3061.877
2023-09-27,233867.113,3394.056
2023-09-28,234882.310,3417.868
2023-09-29,234978.186,3505.098
2023-09-30,224529.501,3323.408


In [61]:
# Interval coverage on test data
# ==============================================================================
inside_interval = np.where(
                      (datos.loc[predictions.index, 'Demanda_SIN'] >= predictions['lower_bound']) & \
                      (datos.loc[predictions.index, 'Demanda_SIN'] <= predictions['upper_bound']),
                      True,
                      False
                  )

coverage = inside_interval.mean()
print(f"Coverage of the predicted interval on test data: {100 * coverage}")

Coverage of the predicted interval on test data: 71.42857142857143


In [62]:
from sklearn.metrics import mean_pinball_loss

mean_pinball_loss(df['Demanda_SIN'], df['predictions'])

1179.8581618189285

In [50]:
def winkler_score(y, lower, upper, alpha):
    if y < lower:
        return (upper - lower) + (2 / alpha) * (lower - y)
    elif lower <= y <= upper:
        return (upper - lower)
    elif y > upper:
        return (upper - lower) + (2 / alpha) * (y - upper)
    
    
# Calcular el Winkler Score para cada fila del DataFrame
alpha_value = 0.5
df['Winkler_Score'] = df.apply(lambda row: winkler_score(row['Demanda_SIN'], row['lower_bound'], row['upper_bound'], alpha_value), axis=1)
df['Winkler_Score']

FECHA
2023-09-24    6507.030
2023-09-25    7577.860
2023-09-26    7215.730
2023-09-27   13379.494
2023-09-28    8174.618
2023-09-29    8220.932
2023-09-30    9785.027
Freq: D, Name: Winkler_Score, dtype: float64

In [62]:
def calculate_final_winkler_score(quantile_alpha_2, quantile_1_minus_alpha_2, alpha):
    return (quantile_alpha_2 + quantile_1_minus_alpha_2) / alpha


alpha_over_2_quantile = df['Winkler_Score'].quantile(0.25)
one_minus_alpha_over_2_quantile = df['Winkler_Score'].quantile(0.75)

# Calcular el Winkler Score final
calculate_final_winkler_score(alpha_over_2_quantile, one_minus_alpha_over_2_quantile, alpha_value)


32799.548979630286

## GradientBoostingRegressor, XGBRegressor

In [46]:
# Models to compare, GradientBoostingRegressor, XGBRegressor.............
import random
random.seed(123)

models = [
          GradientBoostingRegressor(random_state=123),
          #XGBRegressor(random_state=123)
          ]

# Hyperparameter to search for each model
param_grids = {
               'GradientBoostingRegressor': {'n_estimators': list(range(20, 200, 8)) ,
                                              'subsample': np.linspace(0.001, 1.0, num=4),
                                              'max_depth': list(range(10, 100, 8))}
                                             # 'min_samples_leaf':list(range(1, 30, 3)),, 'min_samples_split': list(range(2, 8, 1)) 'learning_rate':np.linspace(0.001, 1.0, num=3), 'min_weight_fraction_leaf':np.linspace(0.0, 0.5, num=20), 'min_impurity_decrease':np.linspace(0.0, 1.0, num=20), 'ccp_alpha':np.linspace(0.0, 1.0, num=20), 'validation_fraction':np.linspace(0.0, 1.0, num=10), 'alpha':np.linspace(0.0, 1.0, num=10),},
               #'max_depth': list(range(10, 100, 10))
               #'XGBRegressor':{'n_estimators': list(range(20, 160, 5)), 'max_depth': list(range(5, 100, 5)), 'learning_rate':list(range(0.01, 0.1, 0.2)), 'booster':['gbtree', 'gblinear','dart']} 
               }          
                        
# Lags used as predictors
lags_grid = [6, 7 ,14, 21, 28]

columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = nombres_variables + columnas_tipo_dia

df_results = pd.DataFrame()
for i, model in enumerate(models):
    print(f"-------------------------")
    print(f"Grid search for regressor: {model}")
    print(f"-------------------------")

    forecaster = ForecasterAutoreg(
                     regressor = model,
                     lags      = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
                     transformer_y = StandardScaler()
                 )

    # Regressor hyperparameters
    param_grid = param_grids[list(param_grids)[i]]

    results_grid = grid_search_forecaster(
                       forecaster         = forecaster,
                       y                  = datos.loc[:fin_validacion, 'Demanda_SIN'],
                       exog               = datos.loc[:fin_validacion, variables_exogenas],
                       param_grid         = param_grid,
                       lags_grid          = lags_grid,
                       steps              = len(datos.loc[fin_train:fin_validacion]),
                       refit              = True, #True
                       metric             = ['mean_absolute_percentage_error','mean_squared_error', 'mean_absolute_error', 'mean_squared_log_error'],
                       initial_train_size = len(datos.loc[:fin_train]),
                       fixed_train_size   = True, # False
                       allow_incomplete_fold =True,
                       return_best        = True, #True
                       n_jobs             = 'auto',
                       verbose            = False, #True
                       show_progress      = True
                   )
    
    # Create a column with model name
    results_grid['model'] = list(param_grids)[i]
    
    df_results = pd.concat([df_results, results_grid])

df_results = df_results.sort_values(by='mean_absolute_percentage_error')



# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False, #Incremental
                            allow_incomplete_fold =True,
                            steps              = 7, #len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_absolute_percentage_error', 'mean_squared_error', 'mean_absolute_error', 'mean_squared_log_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )
print(metric)
print(f"Error de backtest_mean_squared_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_absolute_percentage_error: {metric[2]:.2f}")
print(f"Error de backtest_mean_squared_log_error: {metric[3]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
mape_m=mean_absolute_percentage_error(y_true, predicciones['pred'])
print("MAPE_METRICA",mape_m)
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())


#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()

-------------------------
Grid search for regressor: GradientBoostingRegressor(random_state=123)
-------------------------
Number of models compared: 5520.


lags grid:   0%|          | 0/5 [1:16:02<?, ?it/s]


KeyboardInterrupt: 

In [23]:
mape_data_test_XGBRegressor=mean_absolute_percentage_error(datos_test['Demanda_SIN'].values, forecaster.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values)
print("MAPE..........................................................",mape_data_test_XGBRegressor*100)

# Save model
#save_forecaster(forecaster, file_name='Ml_models/forecaster_XGBRegressor.py', verbose=False)

MAPE.......................................................... 1.084324779119682


## SVR, LinearSVR

In [27]:
# Models to compare, SVR, LinearSVR.................
import random
random.seed(123)

models = [
          SVR()
          ]

# Hyperparameter to search for each model
param_grids = {
               'SVR':{'degree':list(range(1, 6)), 'C': np.logspace(-1, 3, num=15), 'epsilon': np.linspace(0.01, 0.5, num=30)}
               }


                        
# Lags used as predictors
lags_grid = [6, 7 ,14, 21, 28]

variables_exogenas =  nombres_variables + columnas_tipo_dia


df_results = pd.DataFrame()
for i, model in enumerate(models):
    print(f"-------------------------")
    print(f"Grid search for regressor: {model}")
    print(f"-------------------------")

    forecaster = ForecasterAutoreg(
                     regressor = model,
                     lags      = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
                     transformer_y = StandardScaler()
                 )

    # Regressor hyperparameters
    param_grid = param_grids[list(param_grids)[i]]

    results_grid = grid_search_forecaster(
                       forecaster         = forecaster,
                       y                  = datos.loc[:fin_validacion, 'Demanda_SIN'],
                       exog               = datos.loc[:fin_validacion, variables_exogenas],
                       param_grid         = param_grid,
                       lags_grid          = lags_grid,
                       steps              = len(datos.loc[fin_train:fin_validacion]),
                       refit              = True, #True
                       metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                       initial_train_size = len(datos.loc[:fin_train]),
                       fixed_train_size   = True, # False
                       allow_incomplete_fold =True,
                       return_best        = True, #True
                       n_jobs             = 'auto',
                       verbose            = False, #True
                       show_progress      = True
                   )
    
    # Create a column with model name
    results_grid['model'] = list(param_grids)[i]
    
    df_results = pd.concat([df_results, results_grid])

df_results = df_results.sort_values(by='mean_squared_error')



# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False,
                            allow_incomplete_fold =True,
                            steps              =7, # len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )
print(metric)
print(f"Error de backtest_mean_squared_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_absolute_percentage_error: {metric[2]:.2f}")
print(f"Error de backtest_mean_squared_log_error: {metric[3]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())


#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()

-------------------------
Grid search for regressor: SVR()
-------------------------
Number of models compared: 11250.


lags grid: 100%|██████████| 5/5 [1:20:46<00:00, 969.33s/it] 


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28] 
  Parameters: {'C': 1000.0, 'degree': 5, 'epsilon': 0.16206896551724137}
  Backtesting metric: 88125475.49178427

Information of backtesting process
----------------------------------
Number of observations used for initial training: 2008
Number of observations used for backtesting: 115
    Number of folds: 17
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-05-31 00:00:00  (n=2008)
    Validation: 2023-06-01 00:00:00 -- 2023-06-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-01 00:00:00 -- 2023-06-07 00:00:00  (n=2015)
    Validation: 2023-06-08 00:00:00 -- 2023-06-14 00:00:00  (n=7)
Fold: 2
    Training:   2017-12-01 00:00:00 -- 2023-06-14

100%|██████████| 17/17 [00:00<00:00, 308.10it/s]


[47007724.90161392, 5434.413269427472, 0.02505516584145998, 0.001018804544675106]
Error de backtest_mean_squared_error: 47007724.90
Error de backtest_mean_absolute_error: 5434.41
Error de backtest_mean_absolute_percentage_error: 0.03
Error de backtest_mean_squared_log_error: 0.00
MAPE: 2.446396759478253
None


In [29]:
mape_data_test=mean_absolute_percentage_error(datos_test['Demanda_SIN'].values, forecaster.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values)
print("MAPE..........................................................",mape_data_test*100)

# Save model
save_forecaster(forecaster, file_name='Ml_models/forecaster_SVR.py', verbose=False)

MAPE.......................................................... 1.27251804613669


In [24]:
# Models to compare LinearSVR.................
import random
random.seed(123)

models = [
          LinearSVR(random_state=123)
          ]

# Hyperparameter to search for each model
param_grids = {
               'LinearSVR':{'epsilon':np.linspace(0.01, 0.5, num=30),'C': np.logspace(-1, 3, num=15)}
               }


                        
# Lags used as predictors
lags_grid = [6, 7 ,14, 21, 28]

variables_exogenas =  nombres_variables + columnas_tipo_dia


df_results = pd.DataFrame()
for i, model in enumerate(models):
    print(f"-------------------------")
    print(f"Grid search for regressor: {model}")
    print(f"-------------------------")

    forecaster = ForecasterAutoreg(
                     regressor = model,
                     lags      = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
                     transformer_y = StandardScaler()
                 )

    # Regressor hyperparameters
    param_grid = param_grids[list(param_grids)[i]]

    results_grid = grid_search_forecaster(
                       forecaster         = forecaster,
                       y                  = datos.loc[:fin_validacion, 'Demanda_SIN'],
                       exog               = datos.loc[:fin_validacion, variables_exogenas],
                       param_grid         = param_grid,
                       lags_grid          = lags_grid,
                       steps              = len(datos.loc[fin_train:fin_validacion]),
                       refit              = True, #True
                       metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                       initial_train_size = len(datos.loc[:fin_train]),
                       fixed_train_size   = True, # False
                       allow_incomplete_fold =True,
                       return_best        = True, #True
                       n_jobs             = 'auto',
                       verbose            = False, #True
                       show_progress      = True
                   )
    
    # Create a column with model name
    results_grid['model'] = list(param_grids)[i]
    
    df_results = pd.concat([df_results, results_grid])

df_results = df_results.sort_values(by='mean_squared_error')



# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False,
                            allow_incomplete_fold =True,
                            steps              =7, # len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )
print(metric)
print(f"Error de backtest_mean_squared_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_absolute_percentage_error: {metric[2]:.2f}")
print(f"Error de backtest_mean_squared_log_error: {metric[3]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())


#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()

-------------------------
Grid search for regressor: LinearSVR(random_state=123)
-------------------------
Number of models compared: 2250.


lags grid: 100%|██████████| 5/5 [08:14<00:00, 98.85s/it] 


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
  Parameters: {'C': 0.372759372031494, 'epsilon': 0.04379310344827586}
  Backtesting metric: 58124428.76591128

Information of backtesting process
----------------------------------
Number of observations used for initial training: 2008
Number of observations used for backtesting: 115
    Number of folds: 17
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-05-31 00:00:00  (n=2008)
    Validation: 2023-06-01 00:00:00 -- 2023-06-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-01 00:00:00 -- 2023-06-07 00:00:00  (n=2015)
    Validation: 2023-06-08 00:00:00 -- 2023-06-14 00:00:00  (n=7)
Fold: 2
    Training:   2017-12-01 00:00:00 -- 2023-06-14 00:00:00  (n=2022)
    Validation: 2023-06-1

100%|██████████| 17/17 [00:00<00:00, 2814.64it/s]


[466346447.47359866, 13251.10994405152, 0.06013676804228571, 0.008850730091262119]
Error de backtest_mean_squared_error: 466346447.47
Error de backtest_mean_absolute_error: 13251.11
Error de backtest_mean_absolute_percentage_error: 0.06
Error de backtest_mean_squared_log_error: 0.01
MAPE: 5.965220313476997
               feature  importance
0                lag_1       0.518
1                lag_2      -0.040
2                lag_3       0.017
3                lag_4       0.018
4                lag_5       0.026
5                lag_6       0.063
6                lag_7       0.443
7                lag_8      -0.137
8                lag_9      -0.153
9               lag_10      -0.041
10              lag_11      -0.045
11              lag_12      -0.090
12              lag_13       0.011
13              lag_14       0.028
14               Covid      -0.064
15        Cambio_nivel      -0.074
16              Tiempo       0.000
17       VV_10_MEDIA_D       0.015
18  VV_10_VECT_MEDIA_D     

In [26]:
#MAPE on data_test.........
mape_data_test=mean_absolute_percentage_error(datos_test['Demanda_SIN'].values, forecaster.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values)
print("MAPE..........................................................",mape_data_test*100)


# Save model
save_forecaster(forecaster, file_name='Ml_models/forecaster_LinearSVR.py', verbose=False)

MAPE.......................................................... 1.211898224839873


In [13]:
# Prediction Intervals Test..............
# Predict intervals for next 7 days, quantiles 10th and 90th
# ==============================================================================
predictions = forecaster.predict_interval(steps=len(datos_test), interval=[10, 90], n_boot=1000, exog=datos_test[variables_exogenas])
predictions

,pred,lower_bound,upper_bound
2023-09-24,203710.940,197621.738,208151.062
2023-09-25,228616.647,221574.224,234081.297
2023-09-26,233782.231,226488.861,239260.128
2023-09-27,235286.791,227261.532,241226.487
2023-09-28,237115.901,228228.871,242528.779
2023-09-29,238007.105,229671.990,243612.715
2023-09-30,223747.977,215501.954,229138.743


## MLPRegressor

In [14]:
# Models to compare, MLPRegressor.............
import random
random.seed(123)

models = [
        MLPRegressor(random_state=123)
          ]

# Hyperparameter to search for each model
param_grids = {
            'MLPRegressor':{'alpha': np.logspace(-5, 0, num=5),'power_t': np.linspace(0.0, 1.0, num=5), 'hidden_layer_sizes': [(30,) , (50,) , (100,)], 'activation': ["relu", "tanh"],'early_stopping':[True, False] }
               }


                        
# Lags used as predictors
lags_grid = [6, 7 ,14, 21, 28]

variables_exogenas =  nombres_variables + columnas_tipo_dia


df_results = pd.DataFrame()
for i, model in enumerate(models):
    print(f"-------------------------")
    print(f"Grid search for regressor: {model}")
    print(f"-------------------------")

    forecaster = ForecasterAutoreg(
                     regressor = model,
                     lags      = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
                     transformer_y = StandardScaler()
                 )

    # Regressor hyperparameters
    param_grid = param_grids[list(param_grids)[i]]

    results_grid = grid_search_forecaster(
                       forecaster         = forecaster,
                       y                  = datos.loc[:fin_validacion, 'Demanda_SIN'],
                       exog               = datos.loc[:fin_validacion, variables_exogenas],
                       param_grid         = param_grid,
                       lags_grid          = lags_grid,
                       steps              = len(datos.loc[fin_train:fin_validacion]),
                       refit              = True, #True
                       metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'],
                       initial_train_size = len(datos.loc[:fin_train]),
                       fixed_train_size   = True, # False
                       allow_incomplete_fold =True,
                       return_best        = True, #True
                       n_jobs             = 'auto',
                       verbose            = False, #True
                       show_progress      = True
                   )
    
    # Create a column with model name
    results_grid['model'] = list(param_grids)[i]
    
    df_results = pd.concat([df_results, results_grid])

df_results = df_results.sort_values(by='mean_squared_error')



# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False,
                            allow_incomplete_fold =True,
                            steps              = 7, #len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )


print(metric)
print(f"Error de backtest_mean_squared_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_absolute_percentage_error: {metric[2]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())


#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()

-------------------------
Grid search for regressor: MLPRegressor(random_state=123)
-------------------------
Number of models compared: 1500.


lags grid:   0%|          | 0/5 [00:00<?, ?it/s]

lags grid: 100%|██████████| 5/5 [54:45<00:00, 657.16s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] 
  Parameters: {'activation': 'relu', 'alpha': 0.0031622776601683794, 'early_stopping': True, 'hidden_layer_sizes': (100,), 'power_t': 0.25}
  Backtesting metric: 31890645.035708196

Information of backtesting process
----------------------------------
Number of observations used for initial training: 2008
Number of observations used for backtesting: 115
    Number of folds: 17
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-05-31 00:00:00  (n=2008)
    Validation: 2023-06-01 00:00:00 -- 2023-06-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-01 00:00:00 -- 2023-06-07 00:00:00  (n=2015)
    Validation: 2023-06-08 00:00:00 -- 2023-06-14 00:00:00  (n=7)
Fold: 2
   

100%|██████████| 17/17 [00:00<00:00, 8496.56it/s]


[34883771.43698272, 4559.167184807095, 0.020771077893984073]
Error de backtest_mean_squared_error: 34883771.44
Error de backtest_mean_absolute_error: 4559.17
Error de backtest_mean_absolute_percentage_error: 0.02
MAPE: 2.0523893332843115
None


In [16]:
#MAPE on data_test.........
mape_data_test=mean_absolute_percentage_error(datos_test['Demanda_SIN'].values, forecaster.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values)
print("MAPE..........................................................",mape_data_test*100)

# Save model
save_forecaster(forecaster, file_name='Ml_models/forecaster_MLPRegressor.py', verbose=False)

MAPE.......................................................... 2.37919072717973


In [17]:
# Gráfico Predicciones del Intervals Forecaster with Boostrap...
# ==============================================================================
df = datos.loc[fin_train:fin_validacion, ['Demanda_SIN']].copy()
df['predictions'] = predicciones['pred']
df['lower_bound'] = predicciones['lower_bound']
df['upper_bound'] = predicciones['upper_bound']

fig = go.Figure()

# Agrega las series de datos con colores específicos

fig.add_trace(go.Scatter(x=df.index, y=df['Demanda_SIN'], mode='lines', name='Real Data', line=dict(color='#bc8f8f')))
fig.add_trace(go.Scatter(x=df.index, y=df['predictions'], mode='lines', name='Predictions', line=dict(color='#276981')))
fig.add_trace(go.Scatter(x=df.index, y=df['lower_bound'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', showlegend=False))
fig.add_trace(go.Scatter(x=df.index, y=df['upper_bound'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', name='Prediction Interval'))

# Estilo y diseño del gráfico
fig.update_layout(
    title='Energy Demand Forecast with Prediction Interval',
    yaxis_title='GW',
    xaxis_title='',
    showlegend=True
)
# Mostrar el gráfico
fig.show()


In [18]:
# Predict 10 different forecasting sequences of len(datos_test) steps each using bootstrapping
# ==============================================================================
boot_predictions = forecaster.predict_bootstrapping(steps=len(datos_test), n_boot=10, exog=datos_test[variables_exogenas])
boot_predictions

,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9
2023-09-24,199675.316,206485.723,201955.691,202711.047,200052.951,191968.641,201573.400,198957.385,204698.343,202300.745
2023-09-25,228746.514,228147.602,216639.202,224638.319,228813.167,220492.377,229018.482,229906.749,229374.895,229263.248
2023-09-26,232763.661,228691.607,226005.782,228248.012,234963.289,224811.339,238104.950,236311.426,228635.171,233845.212
2023-09-27,230763.610,228689.494,229301.547,228864.792,231374.655,229400.954,225044.675,242577.176,230834.169,226270.040
2023-09-28,235077.809,235386.910,232584.436,232868.647,242138.596,227482.421,230851.648,238387.450,236346.292,229930.095
2023-09-29,234256.112,231637.898,233670.545,224887.279,237615.282,229892.821,234643.491,237853.191,235547.793,231085.378
2023-09-30,221715.400,218655.117,217692.693,217309.798,220491.314,217554.957,225412.945,221010.148,221365.425,221469.293


In [23]:
# # Ridge plot of bootstrapping predictions
# # ==============================================================================
# _ = plot_prediction_distribution(boot_predictions, figsize=(7, 4))

In [19]:
# Predict the parameters of a normal distribution for the next 120 steps
# ==============================================================================
predictions = forecaster.predict_dist(steps=len(datos_test), distribution=norm, n_boot=1000, exog=datos_test[variables_exogenas])
predictions

,loc,scale
2023-09-24,201053.555,3006.435
2023-09-25,226989.320,3136.823
2023-09-26,230801.337,3084.707
2023-09-27,229677.345,3445.338
2023-09-28,232483.896,3341.216
2023-09-29,233056.811,3408.847
2023-09-30,219813.940,3271.136


In [20]:
# Interval coverage on test data
# ==============================================================================
inside_interval = np.where(
                      (datos.loc[predicciones.index, 'Demanda_SIN'] >= predicciones['lower_bound']) & \
                      (datos.loc[predicciones.index, 'Demanda_SIN'] <= predicciones['upper_bound']),
                      True,
                      False
                  )

coverage = inside_interval.mean()
print(f"Coverage of the predicted interval on test data: {100 * coverage}")

Coverage of the predicted interval on test data: 81.73913043478261


In [21]:
# Prediction Intervals Test..............
# Predict intervals for next 7 steps, quantiles 10th and 90th
# ==============================================================================
predictions = forecaster.predict_interval(steps=len(datos_test), interval=[10, 90], n_boot=1000, exog=datos_test[variables_exogenas])
predictions

,pred,lower_bound,upper_bound
2023-09-24,201189.169,197518.759,204307.898
2023-09-25,227185.683,223050.794,230681.479
2023-09-26,230897.559,226842.535,234437.011
2023-09-27,229701.997,225699.526,233518.277
2023-09-28,232693.941,228522.597,236487.092
2023-09-29,233183.741,228929.272,237080.483
2023-09-30,220077.917,215799.476,223736.866


## RandomForestRegressor

In [8]:
# Models to compare, RandomForestRegressor............
import random
random.seed(123)

models = [
          RandomForestRegressor(random_state=123), 
          #AdaBoostRegressor(random_state=123)
          ]

# Hyperparameter to search for each model
param_grids = {
               'RandomForestRegressor': {'n_estimators': list(range(25, 85, 5)), 'max_depth': list(range(20, 40, 10))},
              #'AdaBoostRegressor': {'n_estimators': list(range(20, 100))}
               }


                        
# Lags used as predictors
lags_grid = [6, 7 ,14, 21, 28]

variables_exogenas =  nombres_variables + columnas_tipo_dia


df_results = pd.DataFrame()
for i, model in enumerate(models):
    print(f"-------------------------")
    print(f"Grid search for regressor: {model}")
    print(f"-------------------------")

    forecaster = ForecasterAutoreg(
                     regressor = model,
                     lags      = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
                     transformer_y = StandardScaler()
                 )

    # Regressor hyperparameters
    param_grid = param_grids[list(param_grids)[i]]

    results_grid = grid_search_forecaster(
                       forecaster         = forecaster,
                       y                  = datos.loc[:fin_validacion, 'Demanda_SIN'],
                       exog               = datos.loc[:fin_validacion, variables_exogenas],
                       param_grid         = param_grid,
                       lags_grid          = lags_grid,
                       steps              = len(datos.loc[fin_train:fin_validacion]),
                       refit              = True, #True
                       metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                       initial_train_size = len(datos.loc[:fin_train]),
                       fixed_train_size   = True, # False
                       allow_incomplete_fold =True,
                       return_best        = True, #True
                       n_jobs             = 'auto',
                       verbose            = False, #True
                       show_progress      = True
                   )
    
    # Create a column with model name
    results_grid['model'] = list(param_grids)[i]
    
    df_results = pd.concat([df_results, results_grid])

df_results = df_results.sort_values(by='mean_squared_error')



# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False,
                            allow_incomplete_fold =True,
                            steps              = 7, #len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )
print(metric)
print(f"Error de backtest_mean_squared_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_absolute_percentage_error: {metric[2]:.2f}")
print(f"Error de backtest_mean_squared_log_error: {metric[3]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())


#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()

-------------------------
Grid search for regressor: RandomForestRegressor(random_state=123)
-------------------------
Number of models compared: 120.


lags grid:   0%|          | 0/5 [00:00<?, ?it/s]

lags grid: 100%|██████████| 5/5 [20:00<00:00, 240.15s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
  Parameters: {'max_depth': 20, 'n_estimators': 25}
  Backtesting metric: 70507768.83248547

Information of backtesting process
----------------------------------
Number of observations used for initial training: 2008
Number of observations used for backtesting: 115
    Number of folds: 17
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-05-31 00:00:00  (n=2008)
    Validation: 2023-06-01 00:00:00 -- 2023-06-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-01 00:00:00 -- 2023-06-07 00:00:00  (n=2015)
    Validation: 2023-06-08 00:00:00 -- 2023-06-14 00:00:00  (n=7)
Fold: 2
    Training:   2017-12-01 00:00:00 -- 2023-06-14 00:00:00  (n=2022)
    Validation: 2023-06-15 00:00:00 -- 2023-

100%|██████████| 17/17 [00:00<00:00, 5657.63it/s]


[30470867.597209744, 4067.216301498556, 0.018133808263396077, 0.0006139546729574383]
Error de backtest_mean_squared_error: 30470867.60
Error de backtest_mean_absolute_error: 4067.22
Error de backtest_mean_absolute_percentage_error: 0.02
Error de backtest_mean_squared_log_error: 0.00
MAPE: 1.8309289865861549
               feature  importance
0                lag_1       0.106
1                lag_2       0.006
2                lag_3       0.006
3                lag_4       0.005
4                lag_5       0.007
5                lag_6       0.016
6                lag_7       0.681
7                lag_8       0.006
8                lag_9       0.006
9               lag_10       0.005
10              lag_11       0.003
11              lag_12       0.003
12              lag_13       0.006
13              lag_14       0.041
14               Covid       0.000
15        Cambio_nivel       0.000
16              Tiempo       0.012
17       VV_10_MEDIA_D       0.003
18  VV_10_VECT_MEDIA_D    

In [10]:
#MAPE on data_test.........
mape_data_test=mean_absolute_percentage_error(datos_test['Demanda_SIN'].values, forecaster.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values)
print("MAPE..........................................................",mape_data_test*100)


# Save model
save_forecaster(forecaster, file_name='Ml_models/forecaster_RandomForestRegressor.py', verbose=False)

MAPE.......................................................... 1.3372905903706112


In [11]:
# Gráfico Predicciones del Intervals Forecaster with Boostrap...
# ==============================================================================
df = datos.loc[fin_train:fin_validacion, ['Demanda_SIN']].copy()
df['predictions'] = predicciones['pred']
df['lower_bound'] = predicciones['lower_bound']
df['upper_bound'] = predicciones['upper_bound']

fig = go.Figure()

# Agrega las series de datos con colores específicos

fig.add_trace(go.Scatter(x=df.index, y=df['Demanda_SIN'], mode='lines', name='Real Data', line=dict(color='#bc8f8f')))
fig.add_trace(go.Scatter(x=df.index, y=df['predictions'], mode='lines', name='Predictions', line=dict(color='#276981')))
fig.add_trace(go.Scatter(x=df.index, y=df['lower_bound'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', showlegend=False))
fig.add_trace(go.Scatter(x=df.index, y=df['upper_bound'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', name='Prediction Interval'))

# Estilo y diseño del gráfico
fig.update_layout(
    title='Energy Demand Forecast with Prediction Interval',
    yaxis_title='GW',
    xaxis_title='',
    showlegend=True
)
# Mostrar el gráfico
fig.show()


In [12]:
# Prediction Intervals Test..............
# Predict intervals for next 7 steps, quantiles 10th and 90th
# ==============================================================================
predictions = forecaster.predict_interval(steps=len(datos_test), interval=[10, 90], n_boot=1000, exog=datos_test[variables_exogenas])
predictions

,pred,lower_bound,upper_bound
2023-09-24,201478.130,199617.367,203102.344
2023-09-25,227366.050,225691.893,229286.003
2023-09-26,233840.103,231251.952,236069.706
2023-09-27,236405.077,233633.502,238434.417
2023-09-28,238373.706,235883.634,240090.070
2023-09-29,237427.957,235318.052,239106.304
2023-09-30,225001.843,222626.869,226697.187


In [13]:
# Interval coverage on test data
# ==============================================================================
inside_interval = np.where(
                      (datos.loc[predicciones.index, 'Demanda_SIN'] >= predicciones['lower_bound']) & \
                      (datos.loc[predicciones.index, 'Demanda_SIN'] <= predicciones['upper_bound']),
                      True,
                      False
                  )

coverage = inside_interval.mean()
print(f"Coverage of the predicted interval on test data: {100 * coverage}")

Coverage of the predicted interval on test data: 58.26086956521739


### ML Evaluation

In [22]:
Ridge =load_forecaster("Ml_models/forecaster_Ridge.py")
ElasticNet  = load_forecaster("Ml_models/forecaster_ElasticNet.py")
Lasso  = load_forecaster("Ml_models/forecaster_lasso.py")
LinearSVR  = load_forecaster("Ml_models/forecaster_LinearSVR.py")
MLPRegressor  = load_forecaster("Ml_models/forecaster_MLPRegressor.py")
RandomForestRegressor  = load_forecaster("Ml_models/forecaster_RandomForestRegressor.py")
SVR  = load_forecaster("Ml_models/forecaster_SVR.py")
XGBRegressor  = load_forecaster("Ml_models/forecaster_XGBRegressor.py")


ForecasterAutoreg 
Regressor: Ridge(alpha=0.037896169806447236, random_state=123) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] 
Transformer for y: StandardScaler() 
Transformer for exog: None 
Window size: 21 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D', 'TIPO_DIA_1--ENE', 'TIPO_DIA_1--MAY', 'TIPO_DIA_2--ENE', 'TIPO_DIA_20--JUL', 'TIPO_DIA_24--DIC', 'TIPO_DIA_25--DIC', 'TIPO_DIA_31--DIC', 'TIPO_DIA_7--AGO', 'TIPO_DIA_8--DIC', 'TIPO_DIA_DOALF', 'TIPO_DIA_DOALFENE', 'TIPO_DIA_DOMINGO', 'TIPO_DIA_DOVDIC', 'TIPO_DIA_DOVENE', 'TIPO_DIA_DSS', 'TIPO_DIA_JSS', 'TIPO_DIA_JUEVES', 'TIPO_DIA_JUVDIC', 'TIPO_DIA_JUVENE', 'TIPO_DIA_LF', 'TIPO_DIA_LFENE', 'TIPO_DIA_LUNES', 'TIPO_DIA_LUVDIC', 'TIPO_DIA_LUVENE', 'TIPO_DIA_MADLF', 'TIPO_DI

In [17]:
mape_data_test=mean_absolute_percentage_error(datos_test['Demanda_SIN'].values, XGBRegressor.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values)
print("MAPE..........................................................",mape_data_test*100)



MAPE.......................................................... 1.084324779119682


In [23]:
models = {
    'Ridge':Ridge,
    'ElasticNet': ElasticNet,
    'Lasso': Lasso,
    'LinearSVR': LinearSVR,
    'MLPRegressor': MLPRegressor,
    'RandomForestRegressor': RandomForestRegressor,
    'SVR': SVR,
    'XGBRegressor': XGBRegressor
}

In [24]:
def calculate_metrics(model, datos_test, variables_exogenas):
    # Asumiendo que 'model' es un modelo ya entrenado
    predictions = model.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values
    actual_values = datos_test['Demanda_SIN'].values
    
    mape = np.mean(np.abs((actual_values - predictions) / actual_values)) * 100
    mae = mean_absolute_error(actual_values, predictions)
    rmse = np.sqrt(mean_squared_error(actual_values, predictions))
    
    return mape, mae, rmse

In [25]:
for model_name, model in models.items():
    mape_data_test, mae_data_test, rmse_data_test = calculate_metrics(model, datos_test, variables_exogenas)
    print(f"Metrics for {model_name}:")
    print(f"MAPE: {mape_data_test:.4f}%")
    print(f"MAE: {mae_data_test:.3f}")
    print(f"RMSE: {rmse_data_test:.3f}\n")

Metrics for Ridge:
MAPE: 1.016%
MAE: 2355.948
RMSE: 2793.564

Metrics for ElasticNet:
MAPE: 1.034%
MAE: 2416.559
RMSE: 2999.887

Metrics for Lasso:
MAPE: 1.017%
MAE: 2359.716
RMSE: 2798.997

Metrics for LinearSVR:
MAPE: 1.212%
MAE: 2831.653
RMSE: 3295.441

Metrics for MLPRegressor:
MAPE: 2.379%
MAE: 5521.892
RMSE: 6082.823

Metrics for RandomForestRegressor:
MAPE: 1.337%
MAE: 3079.088
RMSE: 3374.579

Metrics for SVR:
MAPE: 1.273%
MAE: 2814.902
RMSE: 3743.540

Metrics for XGBRegressor:
MAPE: 1.084%
MAE: 2499.921
RMSE: 3239.548



* ML: Lasso
* Emsemble Methods: XGBRegressor
* SVM: LinearSVR

In [29]:
# Crear un DataFrame vacío con las fechas
result_df = pd.DataFrame(index=datos_test.index)

# Iterar sobre los modelos y agregar las predicciones al DataFrame
for model_name, model in models.items():
    predictions = model.predict(steps=len(datos_test), exog=datos_test[variables_exogenas]).values
    result_df[model_name] = predictions

# Agregar la columna y_test al DataFrame
result_df['y_test'] = datos_test['Demanda_SIN'].values

pronostico_xm=['198935.479304','225441.965435','231708.272221','232350.914002','232588.136322','233661.526792','220663.551312']
pronostico_xm = [float(value) for value in pronostico_xm]

# Agregar la nueva columna al DataFrame
result_df['Forecast_XM'] = pronostico_xm
result_df

,Ridge,ElasticNet,Lasso,LinearSVR,MLPRegressor,RandomForestRegressor,SVR,XGBRegressor,y_test,Forecast_XM
FECHA,,,,,,,,,,
2023-09-24,205235.520,204421.618,205227.933,203710.940,201189.169,201478.130,212010.073,205392.125,203949.278,198935.479
2023-09-25,230529.435,230022.622,230509.498,228616.647,227185.683,227366.050,231224.480,226785.259,233453.085,225441.965
2023-09-26,233639.402,233468.126,233624.454,233782.231,230897.559,233840.103,237427.095,235643.760,236506.302,231708.272
2023-09-27,234258.077,233993.833,234261.320,235286.791,229701.997,236405.077,238681.947,236966.099,238964.909,232350.914
2023-09-28,235374.062,235362.813,235379.811,237115.901,232693.941,238373.706,239683.239,236882.960,235940.357,232588.136
2023-09-29,235464.273,235578.289,235465.331,238007.105,233183.741,237427.957,239435.776,237034.180,235929.693,233661.527
2023-09-30,225163.333,224764.715,225154.200,223747.977,220077.917,225001.843,227877.438,224359.296,228839.628,220663.551


In [30]:
result_df=result_df[['Ridge','Lasso', 'XGBRegressor','y_test','Forecast_XM']]
result_df.rename(columns={'y_test':'Real Demand', 'Forecast_XM': 'XM Forecast'},inplace=True)
result_df

,Ridge,Lasso,XGBRegressor,Real Demand,XM Forecast
FECHA,,,,,
2023-09-24,205235.520,205227.933,205392.125,203949.278,198935.479
2023-09-25,230529.435,230509.498,226785.259,233453.085,225441.965
2023-09-26,233639.402,233624.454,235643.760,236506.302,231708.272
2023-09-27,234258.077,234261.320,236966.099,238964.909,232350.914
2023-09-28,235374.062,235379.811,236882.960,235940.357,232588.136
2023-09-29,235464.273,235465.331,237034.180,235929.693,233661.527
2023-09-30,225163.333,225154.200,224359.296,228839.628,220663.551


In [34]:
import plotly.express as px

# Asumiendo que 'result_df' contiene los datos
fig = px.line(result_df, x=result_df.index, y=result_df.columns, title='Forecasts vs. Real Demand',
              labels={'index': 'Fecha', 'value': 'Demanda_SIN'},
              color_discrete_sequence=['#276981', '#9e9764', '#642424', '#001a57', '#e59882'])

fig.update_layout(xaxis_title='Date-daily', yaxis_title='Electricity Demand SIN kWh')

fig.show()


In [38]:
import plotly.express as px

# Asumiendo que 'result_df' contiene los datos
fig = px.line(result_df, x=result_df.index, y=result_df.columns, title='Forecasts vs. Real Demand',
              labels={'index': 'Fecha', 'value': 'Demanda_SIN'},
              color_discrete_sequence=['#276981', '#9e9764', '#642424', '#001a57', '#e59882'])

# Define un conjunto de estilos de líneas (line_dash) para cada línea
line_dash_styles = ['solid', 'dash', 'dot', 'dashdot', 'longdash']

# Asigna un estilo de línea diferente a cada traza en el gráfico
for i, trace in enumerate(fig.data):
    trace.line.dash = line_dash_styles[i % len(line_dash_styles)]

fig.update_layout(xaxis_title='Date-daily', yaxis_title='Electricity Demand SIN kWh')

fig.show()


In [77]:
import plotly.express as px

# Asumiendo que 'result_df' contiene los datos
fig = px.line(result_df, x=result_df.index, y=result_df.columns, title='Model vs. actual forecasts',
              labels={'index': 'Fecha', 'value': 'Demanda_SIN'},
              color_discrete_sequence=px.colors.qualitative.Set2)

fig.update_layout(xaxis_title='Date-daily', yaxis_title='Electricity Demand SIN kWh')

fig.show()
